In [16]:
import sys
sys.path.append('../')
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [12]:
from modules import get_movie_names

get_movie_names.fetch_movie_names()
get_movie_names.cleanMovieName()

Fetching 1000 movie names.....
1000 movie names fetched and stored as csv under ../data/raw/unclean_movie_names.csv
Cleaning movie names...
Movie names cleaned and stored under ../data/processed/clean_movie_names.csv


In [13]:
!python3 ../modules/prepare_training_data.py

42303it [00:01, 27823.75it/s]
100%|████████████████████████████████| 42303/42303 [00:00<00:00, 1136690.11it/s]
../modules/prepare_training_data.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmu_selected_data['Genre'] = genres
../modules/prepare_training_data.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmu_selected_data['Genre'] = cmu_selected_data['Genre'].astype(str)
../modules/prepare_training_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

## Wikipedia code goes here

Movie name is - The Godfather (Space is there in words)

Use str.replace(" ","_") before sending request

Example

In [19]:
BASE_DIR='../data/processed/'
movies_list= BASE_DIR + 'clean_movie_names.csv'
BASE_URL='https://en.wikipedia.org/wiki/'

In [52]:
pd.read_csv(movies_list)['text'].nunique()

895

In [62]:
def fetch_movie_data():
    cast=[]
    release_year=[]
    plot=[]
    movies_name=pd.read_csv(movies_list)
    for movie in movies_name['text']:
        print(movie)
        movie=movie.replace(" ","_")
        start_url = requests.get(BASE_URL + movie).content
        soup = BeautifulSoup(start_url, 'html.parser')
        table = soup.find("table",{"class":"infobox vevent"})
        if table is None:
            start_url = requests.get(BASE_URL + movie + '_(film)').content
            soup = BeautifulSoup(start_url, 'html.parser')
            table = soup.find("table",{"class":"infobox vevent"})
        full_plot = []
        try:
            tag  = soup.select_one('#Plot').find_parent('h2').find_next_sibling()
            while tag.name == 'p':
                full_plot.append(tag.text)
                tag = tag.find_next_sibling()
        except:
            print("No Plot Found")
        values = []
        keys=[]
        if table is not None:
            for tr in table:
                th=tr.find_all('th')
                td = tr.find_all('td')
                value = [tr.text for tr in td]
                key= [tr.text for tr in th]
                values.append(value)
                keys.append(key)
            values=values[0]
            keys=keys[0]
            dictionary = dict(zip(keys, values))
            try:
                cast.append(dictionary['Starring'])
                release_year.append(dictionary['Release date'])
                plot.append(full_plot)
            except:
                print("Key Error")
    return pd.DataFrame({'cast':cast,'release_year':release_year,'plot':plot})

In [63]:
data=fetch_movie_data()

The Godfather
The Godfather: Part II
Schindler's List
The Lord of the Rings: The Return of the King
One Flew Over the Cuckoo's Nest
Forrest Gump
The Silence of the Lambs
Casablanca
The Departed
American Beauty
Gladiator
No Plot Found
Lawrence of Arabia
Braveheart
Amadeus
All About Eve
The Sting
Unforgiven
The Apartment
12 Years a Slave
The Bridge on the River Kwai
On the Waterfront
Rebecca
No Plot Found
The Deer Hunter
It Happened One Night
No Country for Old Men
Platoon
Gone with the Wind
Annie Hall
The King's Speech
Million Dollar Baby
Ben-Hur
No Plot Found
The Best Years of Our Lives
Gandhi
A Beautiful Mind
Rocky
Slumdog Millionaire
The ArtistI 
No Plot Found
All Quiet on the Western Front
No Plot Found
Rain Man
The Lost Weekend
SpotlightI 
No Plot Found
In the Heat of the Night
Dances with Wolves
Patton
Midnight Cowboy
The Sound of Music


KeyboardInterrupt: 

,cast,release_year,plot
0,\nMarlon Brando\nAl Pacino\nJames Caan\nRichar...,"\nMarch 14, 1972 (1972-03-14) (Loew's State Th...","[In 1945 New York City, at his daughter Connie..."
